In [ ]:
import os
import pandas as pd
from Bio.Seq import Seq
import numpy as np

In [ ]:
# Set dataset path
data_path = '../data'

In [ ]:
# Integrate all samples
df_raw = pd.DataFrame()

for file_nm in [f for f in os.listdir(os.path.join(data_path, 'raw_data')) if f.endswith('.csv')]:
    df = pd.read_csv(os.path.join(data_path, 'raw_data'))
    # Add 'file_id' column
    df.insert(loc=0, column='file_id', value=file_nm.split('_')[0])
    df_raw = pd.concat([df_raw, df], ignore_index=True)

display(df_raw)

,file_id,genome_id,organism,locus_tag,protein_id,product,locus,strand,dna_seq,dna_len,aa_seq,aa_len,ess
0,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0001,ACY86538.1,thr operon leader peptide,190..255,+,ATGAACCGCATCAGCACCACCACCATTACCACCATCACCATTACCA...,66.0,MNRISTTTITTITITTGNGAG,21.0,0
1,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0002,ACY86539.1,bifunctional aspartokinase I/homeserinedehydro...,337..2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,2463.0,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,820.0,0
2,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0003,ACY86540.1,homoserine kinase,2801..3730,+,ATGGTGAAAGTGTATGCCCCGGCTTCCAGCGCGAACATGAGCGTCG...,930.0,MVKVYAPASSANMSVGFDVLGAAVTPVDGTLLGDVVSVEAADHFRL...,309.0,0
3,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0004,ACY86541.1,threonine synthase,3734..5020,+,ATGAAACTCTATAATCTGAAAGACCATAATGAGCAGGTCAGCTTTG...,1287.0,MKLYNLKDHNEQVSFAQAVTQGLGKQQGLFFPHDLPEFSLTEIDEM...,428.0,0
4,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0005,ACY86542.1,hypothetical protein,5114..5887,-,ATGCTGATTCTGATTTCACCTGCAAAAACGCTTGATTATCAAAGCC...,774.0,MLILISPAKTLDYQSPLATTRYTQPELLDHSQQLIQQARQLSAPQI...,257.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283919,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0913,AVX54582.1,Tetracycline resistance ribosomal protectionpr...,20416..22335,+,ATGAAAATTATTAATATTGGAGTTTTAGCTCATGTTGATGCAGGAA...,1920.0,MKIINIGVLAHVDAGKTTLTESLLYNSGAITELGSVDKGTTRTDNT...,639.0,1
283920,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0918,AVX54581.1,Imidazoleglycerol-phosphate dehydratase,18716..19378,-,ATGACAGAGCAGAAAGCCCTAGTAAAGCGTATTACAAATGAAACCA...,663.0,MTEQKALVKRITNETKIQIAISLKGGPLAIEHSIFPEKEAEAVAEQ...,220.0,1
283921,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0930,AVX54826.1,50S ribosomal protein L33,319048..319191,-,ATGCGTGAAAAATACATTTTACGTTGTACAGTTTGTAAAAATGAAA...,144.0,MREKYILRCTVCKNENYIGKNDKKKPKIEVSKYCSNCNKHETHKQKK,47.0,1
283922,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0931,AVX54860.1,Adenylyl-sulfate kinase,360403..361011,+,ATGGCTACTAATATTACTTGGCATCCAAATCTTACTTACGACGAAC...,609.0,MATNITWHPNLTYDERKALRKQDGCTIWLTGLSASGKSTIACALEQ...,202.0,1


In [3]:
# Check NA values
print(df_raw['locus_tag'].isnull().values.any())
print(df_raw['dna_seq'].isnull().values.any())
print(df_raw['dna_len'].isnull().values.any())
print(df_raw['aa_seq'].isnull().values.any())
print(df_raw['aa_len'].isnull().values.any())

False
False
True
True
True


In [4]:
# Select DNA sequences without 'aa_seq'
dna_wo_aa = df_raw.loc[df_raw['aa_seq'].isnull(), 'dna_seq']

# Convert selected DNA sequences to Seq object
dna_wo_aa = list(map(Seq, dna_wo_aa))

# Translate the DNA sequences to AA sequences
aa_seq = list(map(lambda x: x.translate(to_stop=False, stop_symbol="X"), dna_wo_aa))
print(aa_seq[0:5])

# Remove the last stop symbols only
aa_seq = [aa[:-1] if aa[-1:] == "X" else aa for aa in aa_seq]
print(aa_seq[0:5])

# Convert to string
aa_seq = list(map(str, aa_seq))

# Insert the translated sequences into existing dataframe
df_all = df_raw.copy()
df_all.loc[df_all['aa_seq'].isnull(), 'aa_seq'] = aa_seq

df_all['aa_len'] = df_all['aa_seq'].str.len()
df_all['dna_len'] = df_all['dna_seq'].str.len()

# Check NA of 'aa_seq'
print(df_all['aa_seq'].isnull().values.any())
print(df_all['aa_len'].isnull().values.any())


display(df_all)

/usr/local/lib/python3.8/dist-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


[Seq('VDENLGALSLTLSPQELTAIEAVFPHDAAAGPRYWPEIMSTLNRX'), Seq('MTLPFKPHLIALVCSAGLFAASGVLYVKSCAPEAPAQAAAPASEPTQTAVPAPV...SPL'), Seq('MSGTYNATIRRVVVSAWIGNSIEYYDFLLYGLASALVFGPLFFPGASPLTATLS...FAV'), Seq('MVRTAILSVAAACTLFALIGCNNRAEVDALQPAQAAELKPMQQSWRGVLPCADC...THX'), Seq('MGNYIRPLSDVVFSIASDNLWIEDSAIQQLYTTAKLTGMKRVIGMPDLHPGRGY...KAH')]
[Seq('VDENLGALSLTLSPQELTAIEAVFPHDAAAGPRYWPEIMSTLNR'), Seq('MTLPFKPHLIALVCSAGLFAASGVLYVKSCAPEAPAQAAAPASEPTQTAVPAPV...SPL'), Seq('MSGTYNATIRRVVVSAWIGNSIEYYDFLLYGLASALVFGPLFFPGASPLTATLS...FAV'), Seq('MVRTAILSVAAACTLFALIGCNNRAEVDALQPAQAAELKPMQQSWRGVLPCADC...CTH'), Seq('MGNYIRPLSDVVFSIASDNLWIEDSAIQQLYTTAKLTGMKRVIGMPDLHPGRGY...KAH')]
False
False


,file_id,genome_id,organism,locus_tag,protein_id,product,locus,strand,dna_seq,dna_len,aa_seq,aa_len,ess
0,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0001,ACY86538.1,thr operon leader peptide,190..255,+,ATGAACCGCATCAGCACCACCACCATTACCACCATCACCATTACCA...,66,MNRISTTTITTITITTGNGAG,21,0
1,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0002,ACY86539.1,bifunctional aspartokinase I/homeserinedehydro...,337..2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,2463,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,820,0
2,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0003,ACY86540.1,homoserine kinase,2801..3730,+,ATGGTGAAAGTGTATGCCCCGGCTTCCAGCGCGAACATGAGCGTCG...,930,MVKVYAPASSANMSVGFDVLGAAVTPVDGTLLGDVVSVEAADHFRL...,309,0
3,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0004,ACY86541.1,threonine synthase,3734..5020,+,ATGAAACTCTATAATCTGAAAGACCATAATGAGCAGGTCAGCTTTG...,1287,MKLYNLKDHNEQVSFAQAVTQGLGKQQGLFFPHDLPEFSLTEIDEM...,428,0
4,C050,CP001363,Salmonella enterica subsp. enterica serovar Ty...,STM14_0005,ACY86542.1,hypothetical protein,5114..5887,-,ATGCTGATTCTGATTTCACCTGCAAAAACGCTTGATTATCAAAGCC...,774,MLILISPAKTLDYQSPLATTRYTQPELLDHSQQLIQQARQLSAPQI...,257,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283919,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0913,AVX54582.1,Tetracycline resistance ribosomal protectionpr...,20416..22335,+,ATGAAAATTATTAATATTGGAGTTTTAGCTCATGTTGATGCAGGAA...,1920,MKIINIGVLAHVDAGKTTLTESLLYNSGAITELGSVDKGTTRTDNT...,639,1
283920,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0918,AVX54581.1,Imidazoleglycerol-phosphate dehydratase,18716..19378,-,ATGACAGAGCAGAAAGCCCTAGTAAAGCGTATTACAAATGAAACCA...,663,MTEQKALVKRITNETKIQIAISLKGGPLAIEHSIFPEKEAEAVAEQ...,220,1
283921,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0930,AVX54826.1,50S ribosomal protein L33,319048..319191,-,ATGCGTGAAAAATACATTTTACGTTGTACAGTTTGTAAAAATGAAA...,144,MREKYILRCTVCKNENYIGKNDKKKPKIEVSKYCSNCNKHETHKQKK,47,1
283922,O046,CP016816,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0931,AVX54860.1,Adenylyl-sulfate kinase,360403..361011,+,ATGGCTACTAATATTACTTGGCATCCAAATCTTACTTACGACGAAC...,609,MATNITWHPNLTYDERKALRKQDGCTIWLTGLSASGKSTIACALEQ...,202,1


In [ ]:
# Save the result
df_all.to_csv(os.path.join(data_path, "data-seq_raw.csv"), index=False)